In [9]:
import torch
import torch.nn as nn
import math
model = nn.Sequential(
    nn.Linear(42, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 256),
    nn.ReLU(),
    nn.Linear(256, 3),
    nn.Sigmoid(),
)

def positional_encoding(x, L=10):
    freqs = 2.0 ** torch.arange(L).float().to(x.device)
    x_input = x.unsqueeze(-1) * freqs * 2 * torch.pi
    encoding = torch.cat([torch.sin(x_input), torch.cos(x_input)], dim=-1)
    encoding = torch.cat([x, encoding.reshape(*x.shape[:-1], -1)], dim=-1) # add to original input    
    return encoding

loss_fn=nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
epochs=20
batch_size=10000

from torch.utils.data import Dataset, DataLoader
import numpy as np
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, image_path, N):
        self.image = np.array(Image.open(image_path))
        self.height, self.width, _ = self.image.shape
        self.N = N

    def __len__(self):
        return self.height * self.width // self.N

    def __getitem__(self, idx):
        # Randomly sample N pixels
        idxs = np.random.choice(self.height * self.width, self.N)

        # Convert to 2D coordinates
        coords = np.array([divmod(idx, self.width) for idx in idxs])

        # Normalize coordinates
        coords = coords / np.array([self.height, self.width])

        # Get colors and normalize
        colors = self.image[coords[:, 0], coords[:, 1]] / 255.0

        return torch.tensor(coords, dtype=torch.float32), torch.tensor(colors, dtype=torch.float32)

dataset = ImageDataset("C:\Users\kamle\Downloads\test.jpg", N=10000)
dataloader = DataLoader(dataset, batch_size=10000)
# Training loop
def train_model(model, image, optimizer, criterion, iters=3000, batch_size=1000, device='cpu'):
    model.train()
    h, w, c = image.shape
    
    x = torch.linspace(0, w-1, w).repeat(h, 1).to(device) / (w)  # Normalize by dividing by width
    y = torch.linspace(0, h-1, h).repeat(w, 1).transpose(0, 1).to(device) / (h)  # Normalize by dividing by height
    all_coords = torch.stack([x, y], dim=-1).view(-1, 2)

    all_pixels = torch.from_numpy(image).view(-1, 3).to(device)
    
    for iteration in range(iters):
        idx = torch.randint(0, h*w, (batch_size,), device=device)
        coords_batch = all_coords.reshape(-1, 2)[idx]
        pixel_batch = all_pixels[idx]
        
        optimizer.zero_grad()
        outputs = model(coords_batch)
        loss = criterion(outputs, pixel_batch)
        loss.backward()
        optimizer.step()

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3441270581.py, line 55)

In [7]:
image = np.array(Image.open("C:/Users/kamle/Downloads/1706544850231.jpg")) / 255.0
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

# Train model
train_model(model, image, optimizer, loss_fn, iters=epochs, batch_size=1000, device='cpu')

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1000x2 and 42x256)